In [14]:
import basedosdados as bd
import pymove as pm

In [5]:
gtfs_tables = ['shapes', 'trips', 'stop_sequences']
gtfs = dict()
for tb in gtfs_tables:
    gtfs[tb] = bd.read_table(
        dataset_id='br_rj_riodejaneiro_gtfs',
        table_id=tb,
        query_project_id='rj-smtr',
        billing_project_id='rj-smtr'
    )

Downloading: 100%|██████████| 870/870 [00:03<00:00, 260.01rows/s]


In [6]:
gtfs['trips']

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,46,S,46_0_S,PENHA,0,,710004600
1,46,U,46_0_U,PENHA,0,,710004600
2,38A,D,38A_0_D,GALEÃO,0,,731003800
3,38A,S,38A_0_S,GALEÃO,0,,731003800
4,38A,U,38A_0_U,GALEÃO,0,,731003800
...,...,...,...,...,...,...,...
95,51A,U,51A_1_U,TERMINAL ALVORADA,1,,731005110
96,53A,S,53A_1_S,TERMINAL ALVORADA,1,,731005310
97,53A,U,53A_1_U,TERMINAL ALVORADA,1,,731005310
98,42A,S,42A_1_S,MADUREIRA MANACEIA,1,,731004210


In [7]:
gtfs['shapes']

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,710001000,-22936.0,-43666754.0,672
1,710001000,-22936.0,-43666754.0,673
2,710001100,-22936.0,-43666754.0,778
3,710001100,-22936.0,-43666754.0,779
4,710001400,-22936.0,-43666754.0,500
...,...,...,...,...
33265,731003810,-22876159.0,-43335464.0,694
33266,731004210,-22876159.0,-43335464.0,780
33267,710004610,-22876159.0,-43335464.0,375
33268,710001100,-23019519.0,-43487182.0,321


In [ ]:
## Testing pymove

In [65]:
gps = bd.read_sql(
    """
    SELECT * 
    FROM `rj-smtr.dashboard_monitoramento_brt.registros_filtrada` 
    WHERE data = DATE '2021-03-30'
    AND placa is not null
    """,
    billing_project_id='rj-smtr'
)

Downloading: 100%|██████████| 114987/114987 [00:05<00:00, 21388.88rows/s]


In [66]:
gps.columns

Index(['codigo', 'placa', 'linha', 'latitude', 'longitude', 'timestamp_gps',
       'velocidade', 'id_migracao_trajeto', 'sentido', 'trajeto',
       'timestamp_captura', 'data', 'hora'],
      dtype='object')

In [105]:
gps.head()

,codigo,placa,linha,latitude,longitude,timestamp_gps,velocidade,id_migracao_trajeto,sentido,trajeto,timestamp_captura,data,hora
0,12739,LMJ4F12,14,-23.000070,-43.400967,2021-03-30 00:07:02,52,138,ida,14 - CAMPO GRANDE X SALVADOR ALLENDE,2021-03-30 00:07:13.161925,2021-03-30,0
1,13420,KWI9708,38A,-22.926362,-43.373737,2021-03-30 00:00:18,37,7029,volta,38A - GALEÃO X ALVORADA (PARADOR),2021-03-30 00:00:39.307204,2021-03-30,0
2,30539,KXE7726,38A,-22.839825,-43.239530,2021-03-30 00:00:01,0,7029,volta,38A - GALEÃO X ALVORADA (PARADOR),2021-03-30 00:00:39.307204,2021-03-30,0
3,12739,LMJ4F12,14,-23.000893,-43.416022,2021-03-30 00:00:02,38,138,ida,14 - CAMPO GRANDE X SALVADOR ALLENDE,2021-03-30 00:00:39.307204,2021-03-30,0
4,44706,LRG8855,31,-22.854400,-43.252003,2021-03-30 00:35:26,0,3061,ida,31 – ALVORADA X VICENTE DE CARVALHO ( Semi dir...,2021-03-30 00:36:31.155501,2021-03-30,0


In [67]:
mdf = pm.MoveDataFrame(data=gps[['latitude', 'longitude', 'timestamp_captura', 'placa']],
                 latitude='latitude',
                 longitude='longitude',
                 datetime='timestamp_captura',
                 traj_id='placa'
                )

In [68]:
mdf.show_trajectories_info()


====================== INFORMATION ABOUT DATASET ======================

Number of Points: 114987

Number of IDs objects: 192

Start Date:2021-03-30 00:00:39.307204     End Date:2021-03-30 23:59:30.461471

Bounding Box:(-23.021778333333334, -43.6967873, -22.8077171, -43.1697711)





In [69]:
mdf.generate_date_features()
mdf.generate_hour_features()

Creating date features...
..Date features was created...


Creating or updating a feature for hour...

...Hour feature was created...



In [70]:
mdf.head()

,lat,lon,datetime,id,date,hour
0,-23.000070,-43.400967,2021-03-30 00:07:13.161925,LMJ4F12,2021-03-30,0
1,-22.926362,-43.373737,2021-03-30 00:00:39.307204,KWI9708,2021-03-30,0
2,-22.839825,-43.239530,2021-03-30 00:00:39.307204,KXE7726,2021-03-30,0
3,-23.000893,-43.416022,2021-03-30 00:00:39.307204,LMJ4F12,2021-03-30,0
4,-22.854400,-43.252003,2021-03-30 00:36:31.155501,LRG8855,2021-03-30,0


In [71]:
mdf.generate_dist_time_speed_features()

...Sorting by id and datetime to increase performance

...Set id as index to a higher performance


Creating or updating distance, time and speed features in meters by seconds



...Reset index...



In [72]:
mdf.head()

,id,lat,lon,datetime,date,hour,dist_to_prev,time_to_prev,speed_to_prev
0,KOP5438,-22.854203,-43.251677,2021-03-30 00:02:41.106991,2021-03-30,0,NaN,NaN,NaN
1,KOP5438,-22.854203,-43.251677,2021-03-30 00:32:27.816715,2021-03-30,0,0.0,1786.709724,0.0
2,KOP5438,-22.854203,-43.251677,2021-03-30 01:02:16.283130,2021-03-30,1,0.0,1788.466415,0.0
3,KOP5438,-22.854203,-43.251677,2021-03-30 01:32:33.123508,2021-03-30,1,0.0,1816.840378,0.0
4,KOP5438,-22.854203,-43.251677,2021-03-30 02:02:21.173259,2021-03-30,2,0.0,1788.049751,0.0


In [73]:
len(mdf)

114987

In [74]:
from pymove import filters

In [75]:
filters.clean_consecutive_duplicates(mdf, subset=['lat', 'lon'])

,id,lat,lon,datetime,date,hour,dist_to_prev,time_to_prev,speed_to_prev
0,KOP5438,-22.854203,-43.251677,2021-03-30 00:02:41.106991,2021-03-30,0,NaN,NaN,NaN
5,KOP5438,-22.853983,-43.251428,2021-03-30 02:31:28.565671,2021-03-30,2,35.297517,1747.392412,0.020200
8,KOP5438,-22.854350,-43.251670,2021-03-30 03:08:33.253232,2021-03-30,3,47.702169,367.358643,0.129852
16,KOP5438,-22.854555,-43.251167,2021-03-30 03:16:35.920325,2021-03-30,3,56.387225,61.460902,0.917449
17,KOP5438,-22.854722,-43.250707,2021-03-30 03:17:36.931336,2021-03-30,3,50.646555,61.011011,0.830122
...,...,...,...,...,...,...,...,...,...
114788,RKP0B06,-23.000860,-43.363767,2021-03-30 17:03:33.530968,2021-03-30,17,1.566046,61.288739,0.025552
114789,RKP0B06,-23.000863,-43.363777,2021-03-30 17:04:33.409556,2021-03-30,17,1.076536,59.878588,0.017979
114802,RKP0B06,-23.000824,-43.363721,2021-03-30 17:17:11.314197,2021-03-30,17,7.174125,61.172095,0.117278
114803,RKP0B06,-23.000855,-43.363701,2021-03-30 17:18:12.315295,2021-03-30,17,3.992816,61.001098,0.065455


In [76]:
filters.clean_gps_nearby_points_by_distances(mdf, radius_area = 10)


Cleaning gps points from radius of 10 meters

...Dropping 51409 rows of gps points

...Rows before: 114987, Rows after:63578, Sum drop:51409

51409 GPS points were dropped


,id,lat,lon,datetime,date,hour,dist_to_prev,time_to_prev,speed_to_prev
0,KOP5438,-22.854203,-43.251677,2021-03-30 00:02:41.106991,2021-03-30,0,NaN,NaN,NaN
5,KOP5438,-22.853983,-43.251428,2021-03-30 02:31:28.565671,2021-03-30,2,35.297517,1747.392412,0.020200
8,KOP5438,-22.854350,-43.251670,2021-03-30 03:08:33.253232,2021-03-30,3,47.702169,367.358643,0.129852
16,KOP5438,-22.854555,-43.251167,2021-03-30 03:16:35.920325,2021-03-30,3,56.387225,61.460902,0.917449
17,KOP5438,-22.854722,-43.250707,2021-03-30 03:17:36.931336,2021-03-30,3,50.646555,61.011011,0.830122
...,...,...,...,...,...,...,...,...,...
114764,RKP0B06,-22.998231,-43.363759,2021-03-30 16:39:19.368605,2021-03-30,16,292.794563,123.152839,2.377489
114765,RKP0B06,-22.997846,-43.363622,2021-03-30 16:40:21.178813,2021-03-30,16,45.054594,61.810208,0.728918
114784,RKP0B06,-22.999405,-43.361943,2021-03-30 16:59:32.821341,2021-03-30,16,241.744429,61.139847,3.953959
114785,RKP0B06,-22.999839,-43.369093,2021-03-30 17:00:30.486646,2021-03-30,17,733.346220,57.665305,12.717287


In [77]:
filters.clean_gps_nearby_points_by_speed(mdf, speed_radius=3.0)


Cleaning gps points using 3.0 speed radius

...Dropping 66755 rows of gps points

...Rows before: 114987, Rows after:48232, Sum drop:66755

66755 GPS points were dropped


,id,lat,lon,datetime,date,hour,dist_to_prev,time_to_prev,speed_to_prev
0,KOP5438,-22.854203,-43.251677,2021-03-30 00:02:41.106991,2021-03-30,0,NaN,NaN,NaN
60,KOP5438,-22.855538,-43.255533,2021-03-30 05:09:12.959307,2021-03-30,5,262.191661,60.344877,4.344887
61,KOP5438,-22.857457,-43.257587,2021-03-30 05:10:13.223649,2021-03-30,5,299.609379,60.264342,4.971586
62,KOP5438,-22.853443,-43.261667,2021-03-30 05:11:14.697649,2021-03-30,5,611.490830,61.474000,9.947146
63,KOP5438,-22.849718,-43.264428,2021-03-30 05:12:15.713432,2021-03-30,5,501.638428,61.015783,8.221454
...,...,...,...,...,...,...,...,...,...
114717,RKP0B06,-23.002625,-43.365590,2021-03-30 15:51:17.263678,2021-03-30,15,277.135905,57.572801,4.813660
114718,RKP0B06,-23.000879,-43.363803,2021-03-30 15:52:18.559623,2021-03-30,15,266.714240,61.295945,4.351254
114784,RKP0B06,-22.999405,-43.361943,2021-03-30 16:59:32.821341,2021-03-30,16,241.744429,61.139847,3.953959
114785,RKP0B06,-22.999839,-43.369093,2021-03-30 17:00:30.486646,2021-03-30,17,733.346220,57.665305,12.717287


In [78]:
from pymove import stay_point_detection

In [79]:
stay_point_detection.create_or_update_datetime_in_format_cyclical(mdf)

Encoding cyclical continuous features - 24-hour time
...hour_sin and  hour_cos features were created...



In [81]:
mdf.head()

,id,lat,lon,datetime,date,hour,dist_to_prev,time_to_prev,speed_to_prev,hour_sin,hour_cos
0,KOP5438,-22.854203,-43.251677,2021-03-30 00:02:41.106991,2021-03-30,0,NaN,NaN,NaN,0.000000,1.000000
1,KOP5438,-22.854203,-43.251677,2021-03-30 00:32:27.816715,2021-03-30,0,0.0,1786.709724,0.0,0.000000,1.000000
2,KOP5438,-22.854203,-43.251677,2021-03-30 01:02:16.283130,2021-03-30,1,0.0,1788.466415,0.0,0.269797,0.962917
3,KOP5438,-22.854203,-43.251677,2021-03-30 01:32:33.123508,2021-03-30,1,0.0,1816.840378,0.0,0.269797,0.962917
4,KOP5438,-22.854203,-43.251677,2021-03-30 02:02:21.173259,2021-03-30,2,0.0,1788.049751,0.0,0.519584,0.854419


In [82]:
stay_point_detection.create_or_update_move_stop_by_dist_time(mdf, dist_radius=40, time_radius=1000)

Split trajectories by max distance between adjacent points: 40
...setting id as index


... Reseting index

...No trajs with only one point. (114987, 12)
...Sorting by id and datetime to increase performance

...Set id as index to a higher performance


Creating or updating distance, time and speed features in meters by seconds



...Reset index...

***********************
by_max_dist took 05.17s
***********************
...Sorting by segment_stop and datetime to increase performance

...Set segment_stop as index to a higher performance


Creating or updating distance, time and speed features in meters by seconds



...Reset index...

Create or update stop as True or False
...Creating stop features as True or False using 1000 to time in seconds
False    74356
True     40631
Name: stop, dtype: int64
*******************************************************
create_or_update_move_stop_by_dist_time took 04m:49.52s
*******************************************************


In [84]:
mdf

,segment_stop,id,lat,lon,datetime,date,hour,dist_to_prev,time_to_prev,speed_to_prev,hour_sin,hour_cos,stop
0,1,KOP5438,-22.854203,-43.251677,2021-03-30 00:02:41.106991,2021-03-30,0,NaN,NaN,NaN,0.000000,1.000000,True
1,1,KOP5438,-22.854203,-43.251677,2021-03-30 00:32:27.816715,2021-03-30,0,0.0,1786.709724,0.0,0.000000,1.000000,True
2,1,KOP5438,-22.854203,-43.251677,2021-03-30 01:02:16.283130,2021-03-30,1,0.0,1788.466415,0.0,0.269797,0.962917,True
3,1,KOP5438,-22.854203,-43.251677,2021-03-30 01:32:33.123508,2021-03-30,1,0.0,1816.840378,0.0,0.269797,0.962917,True
4,1,KOP5438,-22.854203,-43.251677,2021-03-30 02:02:21.173259,2021-03-30,2,0.0,1788.049751,0.0,0.519584,0.854419,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114982,58425,RKP0B06,-23.000846,-43.363712,2021-03-30 20:17:25.851587,2021-03-30,20,0.0,59.574180,0.0,-0.730836,0.682553,True
114983,58425,RKP0B06,-23.000846,-43.363712,2021-03-30 20:18:27.139160,2021-03-30,20,0.0,61.287573,0.0,-0.730836,0.682553,True
114984,58425,RKP0B06,-23.000846,-43.363712,2021-03-30 20:19:28.164075,2021-03-30,20,0.0,61.024915,0.0,-0.730836,0.682553,True
114985,58425,RKP0B06,-23.000846,-43.363712,2021-03-30 20:20:29.003143,2021-03-30,20,0.0,60.839068,0.0,-0.730836,0.682553,True


In [85]:
from pymove import compression

In [87]:
compressed_mdf = compression.compress_segment_stop_to_point(mdf)

...setting mean to lat and lon...
...get only segments stop...


...Dropping 38249 points...
...Shape_before: 114987
...Current shape: 76738
******************************************
compress_segment_stop_to_point took 27.48s
******************************************


In [88]:
compressed_mdf.generate_dist_time_speed_features()

...Sorting by id and datetime to increase performance

...Set id as index to a higher performance


Creating or updating distance, time and speed features in meters by seconds



...Reset index...



In [89]:
compressed_mdf

,id,segment_stop,lat,lon,datetime,date,hour,dist_to_prev,time_to_prev,speed_to_prev,hour_sin,hour_cos,stop,lat_mean,lon_mean
0,KOP5438,1,-22.854203,-43.251677,2021-03-30 00:02:41.106991,2021-03-30,0,NaN,NaN,NaN,0.000000,1.000000,True,-22.854203,-43.251677
1,KOP5438,1,-22.853983,-43.251428,2021-03-30 03:02:25.894589,2021-03-30,3,35.297517,10784.787598,0.003273,0.730836,0.682553,True,-22.854203,-43.251677
2,KOP5438,2,-22.854350,-43.251670,2021-03-30 03:08:33.253232,2021-03-30,3,47.702169,367.358643,0.129852,0.730836,0.682553,False,NaN,NaN
3,KOP5438,2,-22.854350,-43.251670,2021-03-30 03:09:34.190854,2021-03-30,3,0.000000,60.937622,0.000000,0.730836,0.682553,False,NaN,NaN
4,KOP5438,2,-22.854350,-43.251670,2021-03-30 03:10:37.173883,2021-03-30,3,0.000000,62.983029,0.000000,0.730836,0.682553,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76733,RKP0B06,58422,-22.997856,-43.363600,2021-03-30 16:58:31.681494,2021-03-30,16,2.447393,1090.502681,0.002244,-0.942261,-0.334880,True,-22.997856,-43.363600
76734,RKP0B06,58423,-22.999405,-43.361943,2021-03-30 16:59:32.821341,2021-03-30,16,241.744429,61.139847,3.953959,-0.942261,-0.334880,False,NaN,NaN
76735,RKP0B06,58424,-22.999839,-43.369093,2021-03-30 17:00:30.486646,2021-03-30,17,733.346220,57.665305,12.717287,-0.997669,-0.068242,False,NaN,NaN
76736,RKP0B06,58425,-23.000904,-43.363760,2021-03-30 17:01:30.932279,2021-03-30,17,558.482918,60.445633,9.239425,-0.997669,-0.068242,True,-23.000846,-43.363712


In [90]:
from pymove.visualization import folium as f

In [104]:
f.plot_stops(filters.by_id(mdf,   "KOP5438"), radius=0)

...Sorting by id and datetime to increase performance

...Set id as index to a higher performance


Creating or updating distance features in meters...



...Reset index...


Creating or updating features MOVE and STOPS...


....There are 173 stops to this parameters

